<a href="https://colab.research.google.com/github/davidvlaminck/AWVGeneric/blob/master/UseCases/GetAssetsByType/GetAssetsByType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pas eerst onderstaande parameter aan en voer het script uit. Een cookie vervalt binnen de 24 uur. Je kan [hier](https://sites.google.com/mow.vlaanderen.be/em-infra/afspraken-conventies/awv-cookie-ophalen) lezen hoe je de AWV ACM cookie kan ophalen.

In [ ]:
awv_acm_cookie = '9888afc27f3046d791bf23b6d0572198'

Voer onderstaande script uit om de GitHub repo te clonen.

In [ ]:
import os, shutil
if os.path.exists('AWVGeneric'):
  shutil.rmtree('AWVGeneric')
! git clone https://github.com/davidvlaminck/AWVGeneric

Cloning into 'AWVGeneric'...
remote: Enumerating objects: 467, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 467 (delta 147), reused 159 (delta 99), pack-reused 243 (from 1)
Receiving objects: 100% (467/467), 882.63 KiB | 3.92 MiB/s, done.
Resolving deltas: 100% (275/275), done.


Voer onderstaande script uit om alle assets van een bepaald type te downloaden. Je kan de typeUri parameter aanpassen, zie https://apps.mow.vlaanderen.be/eminfra/admin/installatietypes

In [ ]:
import sys
sys.path.insert(0, '/content/AWVGeneric')  # adds the Python files from the GitHub repo

import json
from pathlib import Path

from API.EMInfraClient import EMInfraClient
from API.Enums import Environment, AuthType


if __name__ == '__main__':
    eminfra_client = EMInfraClient(env=Environment.PRD, auth_type=AuthType.COOKIE, cookie=awv_acm_cookie)
    generator_assets = eminfra_client.get_objects_from_oslo_search_endpoint(url_part='assets', filter_string={
        "typeUri": 'https://lgc.data.wegenenverkeer.be/ns/installatie#SeinbrugDVM',
        # uri uit https://apps.mow.vlaanderen.be/eminfra/admin/installatietypes/6f66dad8-8290-4d07-8e8b-6add6c7fe723
        'actief': True})

    print('Downloading...')
    all_assets = list(generator_assets)
    with open(Path('seinbruggen.json'), 'w') as json_file:
        json.dump(all_assets, json_file, indent=4)
    print('Done.')

Downloading...
Done.
